# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [114]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [115]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [116]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [117]:
diamonds.shape

(53940, 10)

In [118]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [119]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [120]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/home/james/anaconda3/envs/wandb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:13:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [121]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/home/james/anaconda3/envs/wandb/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Incorporate validation

In [122]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [123]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640


/home/james/anaconda3/envs/wandb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:13:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[20]	train-rmse:491.51435	validation-rmse:544.08058
[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [124]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/home/james/anaconda3/envs/wandb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:13:39] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[102]	train-rmse:376.20429	validation-rmse:532.59813


In [125]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/home/james/anaconda3/envs/wandb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:13:40] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [126]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [127]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [128]:
# Log in to your W&B account
import wandb

wandb.login()

True

In [129]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see

In [130]:
WANDB_PROJECT = 'diamond_prices'

In [131]:
run = wandb.init(project=WANDB_PROJECT, job_type='train-model')

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [132]:
from wandb.xgboost import WandbCallback

In [133]:
xbg_params = {
    "objective": "reg:squarederror",
    "tree_method": "gpu_hist",
    "eval_metric": "rmse",
}

n = 1000
early_stopping_rounds = 20

xgbmodel = xgb.XGBRegressor(**xbg_params,
                            n_estimators=n,
                            early_stopping_rounds=early_stopping_rounds,
                            callbacks=[WandbCallback()])

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
xgbmodel.fit(X_train, y_train, eval_set=[(X_test, y_test)])

[0]	validation_0-rmse:2890.49103
[1]	validation_0-rmse:2151.25657
[2]	validation_0-rmse:1646.16591
[3]	validation_0-rmse:1309.68710
[4]	validation_0-rmse:1077.47338
[5]	validation_0-rmse:916.86440
[6]	validation_0-rmse:826.99597
[7]	validation_0-rmse:760.80217
[8]	validation_0-rmse:719.44433
[9]	validation_0-rmse:693.51193
[10]	validation_0-rmse:672.84882
[11]	validation_0-rmse:654.85984
[12]	validation_0-rmse:634.73614
[13]	validation_0-rmse:623.18225
[14]	validation_0-rmse:617.34221
[15]	validation_0-rmse:613.67147
[16]	validation_0-rmse:607.32331
[17]	validation_0-rmse:604.46119
[18]	validation_0-rmse:598.17601
[19]	validation_0-rmse:594.88868
[20]	validation_0-rmse:593.34843
[21]	validation_0-rmse:590.06852
[22]	validation_0-rmse:587.76214
[23]	validation_0-rmse:585.99364
[24]	validation_0-rmse:582.98290
[25]	validation_0-rmse:582.47269
[26]	validation_0-rmse:580.17669
[27]	validation_0-rmse:579.34976
[28]	validation_0-rmse:578.84001
[29]	validation_0-rmse:577.81118
[30]	validation

/home/james/anaconda3/envs/wandb/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:13:46] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[35]	validation_0-rmse:573.47272
[36]	validation_0-rmse:572.44439
[37]	validation_0-rmse:571.66438
[38]	validation_0-rmse:571.07939
[39]	validation_0-rmse:570.80938
[40]	validation_0-rmse:569.24079
[41]	validation_0-rmse:569.96377
[42]	validation_0-rmse:569.05688
[43]	validation_0-rmse:570.04498
[44]	validation_0-rmse:569.55543
[45]	validation_0-rmse:568.45968
[46]	validation_0-rmse:568.73302
[47]	validation_0-rmse:568.48926
[48]	validation_0-rmse:568.45652
[49]	validation_0-rmse:567.32418
[50]	validation_0-rmse:566.54529
[51]	validation_0-rmse:565.76741
[52]	validation_0-rmse:565.23540
[53]	validation_0-rmse:564.38770
[54]	validation_0-rmse:564.41182
[55]	validation_0-rmse:564.37780
[56]	validation_0-rmse:564.79726
[57]	validation_0-rmse:564.74789
[58]	validation_0-rmse:564.21772
[59]	validation_0-rmse:563.96804
[60]	validation_0-rmse:563.83542
[61]	validation_0-rmse:563.29146
[62]	validation_0-rmse:563.05053
[63]	validation_0-rmse:562.00163
[64]	validation_0-rmse:563.13364
[65]	valid

XGBRegressor(base_score=None, booster=None,
             callbacks=[<wandb.integration.xgboost.xgboost.WandbCallback object at 0x7efa2b7b46d0>],
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=20,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [134]:
run.finish()

best_iteration,▁
best_score,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
validation_0-rmse,█▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,63
best_score,562.00163
epoch,82


In [135]:
sweep_config = {
  "method" : "random",
  "parameters" : {
    "learning_rate" :{
      "min": 0.001,
      "max": 1.0
    },
    "gamma" :{
      "min": 0.001,
      "max": 1.0
    },
    "min_child_weight" :{
      "min": 1,
      "max": 150
    },
    "early_stopping_rounds" :{
      "values" : [10, 20, 30, 40]
    },
    "n_estimators" :{
      "min": 100,
      "max": 1000
    }
  }
}

In [136]:
sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)

Create sweep with ID: 27ja3kk1
Sweep URL: https://wandb.ai/jameszampanu/diamond_prices/sweeps/27ja3kk1


In [137]:
def train():
    diamonds = sns.load_dataset('diamonds')
    diamonds.head()

    X,y = diamonds.drop('price', axis=1), diamonds[['price']]

    # For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

    X['cut'] = X['cut'].astype('category')
    X['color'] = X['color'].astype('category')
    X['clarity'] = X['clarity'].astype('category')
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    with wandb.init(job_type="sweep") as run:
        # bst_params = {
        #     'objective': 'binary:logistic'
        #     , 'base_score': base_rate
        #     , 'gamma': run.config['gamma']
        #     , 'learning_rate': run.config['learning_rate']
        #     , 'max_depth': 3
        #     , 'min_child_weight': run.config['min_child_weight']
        #     , 'n_estimators': 25
        #     , 'nthread': 24 
        #     , 'random_state': 42
        #     , 'reg_alpha': 0
        #     , 'reg_lambda': 0          ## def: 1
        #     , 'eval_metric': ['auc', 'logloss']
        #     , 'tree_method': 'hist' 
        # }
        xbg_params = {
            "objective": "reg:squarederror",
            "tree_method": "gpu_hist",
            "eval_metric": "rmse",
            "learning_rate": run.config['learning_rate'],
            "gamma": run.config['gamma'],
            "min_child_weight": run.config['min_child_weight'],
            "n_estimators": run.config['n_estimators']
        }

        xgbmodel = xgb.XGBRegressor(**xbg_params,
                                    early_stopping_rounds=run.config['early_stopping_rounds'],
                                    callbacks=[WandbCallback()])

        X_train = pd.get_dummies(X_train)
        X_test = pd.get_dummies(X_test)
        xgbmodel.fit(X_train, y_train, eval_set=[(X_test, y_test)])

In [138]:
count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: ot9v5w9y with config:
wandb: 	early_stopping_rounds: 10
wandb: 	gamma: 0.3641507366248487
wandb: 	learning_rate: 0.33237579018613544
wandb: 	min_child_weight: 89
wandb: 	n_estimators: 814
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


[0]	validation_0-rmse:2787.37924
[1]	validation_0-rmse:2001.28527
[2]	validation_0-rmse:1503.05942
[3]	validation_0-rmse:1199.68585
[4]	validation_0-rmse:1013.40151
[5]	validation_0-rmse:890.45731
[6]	validation_0-rmse:817.06260
[7]	validation_0-rmse:763.09086
[8]	validation_0-rmse:732.10232
[9]	validation_0-rmse:706.23621
[10]	validation_0-rmse:683.95971
[11]	validation_0-rmse:671.96324
[12]	validation_0-rmse:660.40692
[13]	validation_0-rmse:653.43501
[14]	validation_0-rmse:642.07463
[15]	validation_0-rmse:635.21923
[16]	validation_0-rmse:630.21488
[17]	validation_0-rmse:624.59925
[18]	validation_0-rmse:622.02216
[19]	validation_0-rmse:614.44228
[20]	validation_0-rmse:612.37611
[21]	validation_0-rmse:611.20840
[22]	validation_0-rmse:607.89371
[23]	validation_0-rmse:604.75843
[24]	validation_0-rmse:602.48516
[25]	validation_0-rmse:599.96402
[26]	validation_0-rmse:600.13168
[27]	validation_0-rmse:599.49497
[28]	validation_0-rmse:596.64100
[29]	validation_0-rmse:595.28317
[30]	validation

[12:14:10] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"



[52]	validation_0-rmse:578.92221
[53]	validation_0-rmse:578.63707
[54]	validation_0-rmse:578.41480
[55]	validation_0-rmse:578.28172
[56]	validation_0-rmse:578.19322
[57]	validation_0-rmse:577.45511
[58]	validation_0-rmse:577.39882
[59]	validation_0-rmse:576.74568
[60]	validation_0-rmse:576.37303
[61]	validation_0-rmse:575.86482
[62]	validation_0-rmse:575.72397
[63]	validation_0-rmse:574.92497
[64]	validation_0-rmse:575.68366
[65]	validation_0-rmse:575.37655
[66]	validation_0-rmse:575.28151
[67]	validation_0-rmse:575.78958
[68]	validation_0-rmse:575.49278
[69]	validation_0-rmse:575.26060
[70]	validation_0-rmse:575.18344
[71]	validation_0-rmse:574.54344
[72]	validation_0-rmse:574.62888
[73]	validation_0-rmse:573.86717
[74]	validation_0-rmse:574.14901
[75]	validation_0-rmse:573.85271
[76]	validation_0-rmse:573.59441
[77]	validation_0-rmse:573.51430
[78]	validation_0-rmse:573.32951
[79]	validation_0-rmse:572.82165
[80]	validation_0-rmse:572.50887
[81]	validation_0-rmse:571.66944
[82]	valid

best_iteration,▁
best_score,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
validation_0-rmse,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,156
best_score,562.19751
epoch,165


...

wandb: Agent Starting Run: fn2kgdzx with config:
wandb: 	early_stopping_rounds: 40
wandb: 	gamma: 0.6888634287335959
wandb: 	learning_rate: 0.35608746223759563
wandb: 	min_child_weight: 130
wandb: 	n_estimators: 165
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


[0]	validation_0-rmse:2705.00555
[1]	validation_0-rmse:1918.41443
[2]	validation_0-rmse:1444.35658
[3]	validation_0-rmse:1147.67310
[4]	validation_0-rmse:967.06011
[5]	validation_0-rmse:857.22615
[6]	validation_0-rmse:786.44682
[7]	validation_0-rmse:748.32142
[8]	validation_0-rmse:721.70404
[9]	validation_0-rmse:701.02452
[10]	validation_0-rmse:686.55274
[11]	validation_0-rmse:678.87151
[12]	validation_0-rmse:667.58731
[13]	validation_0-rmse:661.00687
[14]	validation_0-rmse:652.53022
[15]	validation_0-rmse:647.43868
[16]	validation_0-rmse:642.05802
[17]	validation_0-rmse:638.34950
[18]	validation_0-rmse:634.29562
[19]	validation_0-rmse:629.98982
[20]	validation_0-rmse:627.61914
[21]	validation_0-rmse:624.69378
[22]	validation_0-rmse:623.43006
[23]	validation_0-rmse:621.18687
[24]	validation_0-rmse:616.54582
[25]	validation_0-rmse:615.71854
[26]	validation_0-rmse:613.69654
[27]	validation_0-rmse:611.83358
[28]	validation_0-rmse:611.38531
[29]	validation_0-rmse:609.77744
[30]	validation_

[12:15:13] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"



[34]	validation_0-rmse:601.79194
[35]	validation_0-rmse:601.67264
[36]	validation_0-rmse:601.72188
[37]	validation_0-rmse:601.37024
[38]	validation_0-rmse:601.37268
[39]	validation_0-rmse:599.29704
[40]	validation_0-rmse:597.35913
[41]	validation_0-rmse:596.60793
[42]	validation_0-rmse:596.05912
[43]	validation_0-rmse:593.78245
[44]	validation_0-rmse:591.87748
[45]	validation_0-rmse:591.69227
[46]	validation_0-rmse:591.63379
[47]	validation_0-rmse:591.49182
[48]	validation_0-rmse:590.84644
[49]	validation_0-rmse:589.57515
[50]	validation_0-rmse:589.15932
[51]	validation_0-rmse:588.71589
[52]	validation_0-rmse:588.42634
[53]	validation_0-rmse:587.32954
[54]	validation_0-rmse:587.33372
[55]	validation_0-rmse:587.25075
[56]	validation_0-rmse:586.27620
[57]	validation_0-rmse:586.39036
[58]	validation_0-rmse:585.94247
[59]	validation_0-rmse:586.03169
[60]	validation_0-rmse:585.64391
[61]	validation_0-rmse:585.45541
[62]	validation_0-rmse:584.60715
[63]	validation_0-rmse:583.73732
[64]	valid

best_iteration,▁
best_score,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
validation_0-rmse,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,164
best_score,565.47304
epoch,164


# Experiment Description

- This experiment runs 5 different hyper-parameter configurations using random variations of learning rate, gamma, min child weight, early stopping rounds, num estimators for the XGBRegressor model. Looking at the results of the experiment on WandB we see that sweep run 3 performed best with 10 early stopping rounds, a gamma of 0.35, a learning rate of 0.62, a min child weight of 84, and 754 num estimators.